In [2]:
fetchapi_url = "https://api.github.com/repos/{owner}/{repo}/contents/{path}".format(**{
  "owner": "mikepsinn",
  "repo": "FDAi",
  "path":""
})
fetchapi_url

'https://api.github.com/repos/mikepsinn/FDAi/contents/'

In [1]:
import os, requests
from dotenv import load_dotenv  
load_dotenv('../.env')

True

In [2]:
def fetch_github_repo_contents_urls(
    api_url,path=""):
    headers = {
        "Authorization": f"token {os.environ['GITHUB_TOKEN']}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.get(
        api_url+path, headers=headers
    )
    urls = []
        # Check if the request was successful
    if response.status_code == 200:
        contents = response.json()

        # Ensure contents is a list before iterating
        if not isinstance(contents, list):
            print(f"Expected a list but got {type(contents)}.")
            return urls

        for content in contents:
            print(
                content.get('name', 'No name found'))
            # If it's a directory, recurse into it
            if content.get('type') == 'dir':
                urls.extend(fetch_github_repo_contents_urls(api_url+path, content.get('path')))
            else:
                # Append the file URL to the list
                urls.append(content.get('html_url', 'No URL found'))

    else:
        # Print out the error message from GitHub API
        print(f"Error fetching contents: {response.status_code} - {response.json().get('message')}")

    return urls

In [ ]:
github_urls = fetch_github_repo_contents_urls(
  fetchapi_url
)

In [8]:
print(github_urls)

['https://github.com/mikepsinn/FDAi/blob/develop/.cirun.yml', 'https://github.com/mikepsinn/FDAi/blob/develop/.deepsource.toml', 'https://github.com/mikepsinn/FDAi/blob/develop/.editorconfig', 'https://github.com/mikepsinn/FDAi/blob/develop/.env.example', 'https://github.com/mikepsinn/FDAi/blob/develop/.eslintrc.json', 'https://github.com/mikepsinn/FDAi/blob/develop/.gitattributes', 'https://github.com/mikepsinn/FDAi/blob/develop/.gitignore', 'https://github.com/mikepsinn/FDAi/blob/develop/.idea/rcb-settings.xml', 'https://github.com/mikepsinn/FDAi/blob/develop/.mega-linter.yml', 'https://github.com/mikepsinn/FDAi/blob/develop/.prettierignore', 'https://github.com/mikepsinn/FDAi/blob/develop/.prettierrc', 'https://github.com/mikepsinn/FDAi/blob/develop/.run/api-lib.run.xml', 'https://github.com/mikepsinn/FDAi/blob/develop/.run/api-spec.run.xml', 'https://github.com/mikepsinn/FDAi/blob/develop/.run/start%20api.run.xml', 'https://github.com/mikepsinn/FDAi/blob/develop/.run/swagger.json-c